# Application d'extractions des relations sociales entre entités nommées

- REmplacer les 'her', 'him', 'she', etc... par les noms des entités nommées en utilisant des règles 'à la main'
- Penser aux words-embedding

In [386]:
from stanfordcorenlp import StanfordCoreNLP
from nltk.tokenize import sent_tokenize, word_tokenize

In [387]:
nlp = StanfordCoreNLP('stanford-corenlp-full-2018-10-05')

In [388]:
file = open('Hamlet.txt','r+')
text = file.read()
#text = text.replace("\n"," ")
text = text.replace("’\n","\n")
text_tokens = sent_tokenize(text)
#test = []
#for elt in text_tokens :
#    test.append(word_tokenize(elt))
PERSON = []
DEPENDENCY = [[]]*len(text)
TOKENS = []
RELATIONSHIP = {}
LINKS = ['son','father','mother','daughter','cousin','siblings','husband','wife','spouses','brother','sister','friend','girlfriend','boyfriend']

In [389]:
NE = nlp.ner(text)
parsing = nlp.dependency_parse(text)

for line in text_tokens : 
    TOKENS.append(nlp.word_tokenize(line))
for elt in NE :
    if 'PERSON' in elt :
        PERSON.append(elt[0])

PERSON = list(set(PERSON))
for p in PERSON : 
    RELATIONSHIP[p] = []
    
print(RELATIONSHIP)

{'Claudius': [], 'Gertrude': [], 'Mother': [], 'Hamlet': [], 'Ophelia': [], 'Horatio': []}


In [390]:
nb_line = 0
for line in text_tokens  :
    #print(nb_line,' : ', line)
    parsing = nlp.dependency_parse(line)
    for elt in parsing:
    
        if ('nmod:poss' in elt) :
            #print("Ajout de :", elt, 'à la ligne :', nb_line)
            DEPENDENCY[nb_line] = DEPENDENCY[nb_line] + [(elt)]

        elif('appos' in elt) : 
            #print("Ajout de :", elt, 'à la ligne :', nb_line)
            DEPENDENCY[nb_line] = DEPENDENCY[nb_line] + [(elt)]
    nb_line+=1

In [391]:
nlp.close()

In [392]:
def make_relation(dep,dependencies,nb_line):
    
    if(dep[0]=='nmod:poss') :
        
        #print('make_relation :',dep,dependencies,nb_line)
        
        if (dependencies == []):
            #print('Un seul élément')
            #print(('NaN',dep[1],dep[2],nb_line))
            return(('NaN',dep[1],dep[2],nb_line))
        
        else :
            for elt in dependencies :
                if (elt[0]=='appos'):
                    #print('Appos trouvé')
                    if(dep[1]-elt[1]==0) :
                        
                        #print((elt[2],dep[1],dep[2],nb_line))
                        return((elt[2],dep[1],dep[2],nb_line))
        
    return((0,0,0))
    

In [393]:
#for elt in DEPENDENCY : 
    #print(elt)

On a un problème : 

- On a utilisé la tokenization par phrase de la librairie, c'est à dire qu'on a découpé le texte par phrase avec une liste qui contient dans chacune de ses cases une phrase du texte.

    - Le problème : La fonction n'a pas toujours découpé le texte par phrase (voir par exemple la case numéro 5 dans laquelle on retrouve 2 phrases)



- On a, pour chacune des cases, utilisé l'analyseur de dépendences qui nous permet d'avoir les relations nmod:pos et appos entre les mots pour chacune des phrases qu'on a tokénizé.

    - Le problème est que l'analyseur de dépendences, lui, découpe correctement les phrases. Ainsi, lorsqu'il nous donne l'indice d'un mot, il se base sur son découpage et non celui de la fonction qu'on a utilisé pour découper nos phrases, ce faisant, les indices des mots ne correspondent plus entre notre phrase du texte tokénisé et notre phrase du texte analysé.


Solution temporaire : 

Le problème vient des caractères spéciaux "’" qui encadrent les dialogues, il suffit de les supprimer pour ce texte précis.


In [394]:
nb_line = 0
tmp = []
encoded_relationship = []
id_NaN = 0

for elt in DEPENDENCY :
    try :
        tmp.append(make_relation(elt[0],elt[1:],nb_line))
        nb_line+=1
        
    except : 
        nb_line+=1

for t in tmp : 
    if(t!=(0,0,0)) :
        encoded_relationship.append(t)
        
for r in encoded_relationship :
    
    try :
        line = r[3]
        perso1 = r[0]-1
        lien = r[1]-1
        perso2 = r[2]-1
        print(TOKENS[line][perso1],TOKENS[line][lien],TOKENS[line][perso2])
        
        try :
            RELATIONSHIP[(TOKENS[line][perso1])] += [(TOKENS[line][lien],TOKENS[line][perso2])]
        
        except :
            RELATIONSHIP[(TOKENS[line][perso1])] = [(TOKENS[line][lien],TOKENS[line][perso2])]
        
    
    except :
        
        line = r[3]
        lien = r[1]-1
        perso2 = r[2]-1
        print(r[0],TOKENS[line][lien],TOKENS[line][perso2])
        RELATIONSHIP[str(r[0])+str(id_NaN)] = [(TOKENS[line][lien],TOKENS[line][perso2])]
        
        id_NaN +=1



NaN father Hamlet
Gertrude mother Hamlet
Horatio friend Hamlet
NaN father Hamlet
Ophelia girlfriend his
NaN play their
NaN father Hamlet
Laertes brother Ophelia
NaN drink his
NaN friend his
NaN prince my
{'Claudius': [], 'Gertrude': [('mother', 'Hamlet')], 'Mother': [], 'Hamlet': [], 'Ophelia': [('girlfriend', 'his')], 'Horatio': [('friend', 'Hamlet')], 'NaN0': [('father', 'Hamlet')], 'NaN1': [('father', 'Hamlet')], 'NaN2': [('play', 'their')], 'NaN3': [('father', 'Hamlet')], 'Laertes': [('brother', 'Ophelia')], 'NaN4': [('drink', 'his')], 'NaN5': [('friend', 'his')], 'NaN6': [('prince', 'my')]}


In [404]:
for elt in RELATIONSHIP : 
    print(elt, ":", RELATIONSHIP[elt])

Claudius : []
Gertrude : [('mother', 'Hamlet')]
Mother : []
Hamlet : []
Ophelia : [('girlfriend', 'his')]
Horatio : [('friend', 'Hamlet')]
NaN0 : [('father', 'Hamlet')]
NaN1 : [('father', 'Hamlet')]
NaN2 : [('play', 'their')]
NaN3 : [('father', 'Hamlet')]
Laertes : [('brother', 'Ophelia')]
NaN4 : [('drink', 'his')]
NaN5 : [('friend', 'his')]
NaN6 : [('prince', 'my')]


In [405]:
#dictionnary = {}
#dictionnary['Hamlet'] = [('father','The King'),('Mother','Gertrude')]
#print(dictionnary)
#dictionnary['Hamlet'] += [('lool','ff')]
#print(dictionnary)